# 5章 コンピュータビジョンのためのディープラーニング

- CNNの概要    
- 過学習を抑制するためのデータ拡張    
- 学習済みのCNNを使った特徴

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

# 5.1 - 畳み込みニューラルネットワークの紹介

2章の全結合ネットワークの正解率は97.8%だった。CNNでやると上回るはず。    

今回のCNNはConv2DとMaxPooling2D層



In [0]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Let's display the architecture of our convnet so far:

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


CNNの入力テンソルが(image_height, image_width, image_channels)でバッチ次元を含まない形状。この場合はmnistなので(28, 28, 1)の次元で入力。

畳み込みから全結合に繋ぐ時にflattenで１次元に変換してる。

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

最後ソフトマックス。

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

mnistサンプルを使用する

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 12s 203us/step - loss: 0.1819 - acc: 0.9428
Epoch 2/5
60000/60000 [==============================] - 10s 171us/step - loss: 0.0456 - acc: 0.9859
Epoch 3/5
48512/60000 [=======================>......] - ETA: 1s - loss: 0.0312 - acc: 0.9905

60000/60000 [==============================] - 10s 169us/step - loss: 0.0313 - acc: 0.9905
Epoch 4/5
60000/60000 [==============================] - 10s 170us/step - loss: 0.0237 - acc: 0.9928
Epoch 5/5
60000/60000 [==============================] - 10s 171us/step - loss: 0.0178 - acc: 0.9942


モデルをテストデータセットで評価

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 107us/step


In [11]:
test_acc

0.9902

99.3%の精度でよくなってる。何故よくなったのかCNNの内部を見ていく。